In [1]:
import mturk
import random
from datetime import datetime
import json
from pprint import pprint
import copy
import numpy as np
from collections import OrderedDict
import pandas as pd
import pymongo
import botocore
import uuid
import pdb
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg
import krippendorff_alpha as ka

C:\Users\gabri\anaconda3\envs\phd\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Getting WTR reference and claim data

In [75]:
wtr = pd.read_csv('./WTR/WTR_non_filtered_non_annotated.csv')
wtr_trim = wtr.sample(frac=1, random_state=42).drop_duplicates('reference_id').sort_index().reset_index(drop=True)

wtr_trim = wtr_trim.drop('verb_mock', axis=1)

wtr_trim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   reference_id           648 non-null    object
 1   reference_property_id  648 non-null    object
 2   reference_datatype     648 non-null    object
 3   url                    648 non-null    object
 4   netloc                 648 non-null    object
 5   netloc_agg             648 non-null    object
 6   final_url              648 non-null    object
 7   html                   648 non-null    object
 8   claim_id               648 non-null    object
 9   rank                   648 non-null    object
 10  datatype               648 non-null    object
 11  datavalue              648 non-null    object
 12  entity_id              648 non-null    object
 13  property_id            648 non-null    object
 14  entity_label           648 non-null    object
 15  property_label         

In [76]:
bad_netloc_aggs = [
    'witches.shca.ed.ac.uk', #Single infobox
    'en.isabart.org', #Single infobox
    'bechdeltest.com', #HAS API
    'npg.si.edu', #Image and single infobox
    'www.guidetopharmacology.org', #Single infobox
    'letterboxd.com', #Single infobox and has API
    'www.discogs.com', #Single infobox
    'vocab.getty.edu', #HAS JSON DUMPS
    'www.isfdb.org', #single infobox
    'www.npg.org.uk', #set of infoboxes
    'art.nationalgalleries.org', #image and single infobox
    'www.tate.org.uk', #image and single infobox
    'www.getty.edu', #HAS JSON DUMPS
    'memory-beta.fandom.com', #The portion with the information on Claims is just a long list of names and links
    'www.disease-ontology.org', #A single infobox
    'artgallery.yale.edu', #Image and a single infobox
    'www.imdb.com', #These are author pages and consist of a portrait, an infobox, and lists of movies
    'muckrack.com', #A very short infobox
    'live.dbpedia.org', #It's dbpedia, so it's mainly a huge infobox and there are dumps
    'dbpedia.org' #Same as above
]
wtr_trim_good = wtr_trim[~wtr_trim.netloc_agg.isin(bad_netloc_aggs)].reset_index(drop=True)

wtr_trim_good.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409 entries, 0 to 408
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   reference_id           409 non-null    object
 1   reference_property_id  409 non-null    object
 2   reference_datatype     409 non-null    object
 3   url                    409 non-null    object
 4   netloc                 409 non-null    object
 5   netloc_agg             409 non-null    object
 6   final_url              409 non-null    object
 7   html                   409 non-null    object
 8   claim_id               409 non-null    object
 9   rank                   409 non-null    object
 10  datatype               409 non-null    object
 11  datavalue              409 non-null    object
 12  entity_id              409 non-null    object
 13  property_id            409 non-null    object
 14  entity_label           409 non-null    object
 15  property_label         

In [77]:
for i, row in wtr_trim_good.iterrows():
    r_id = row['reference_id']
    with open(f'./WTR/{r_id}.html', 'w+', encoding='utf8') as f:
        f.write(row['html'])
    row['html'] = f'{r_id}.html'

In [78]:
from ast import literal_eval as leval

wtr_trim_good['datavalue'] = wtr_trim_good['datavalue'].apply(leval)

In [79]:
wtr_trim_good['entity_alias'] = wtr_trim_good['entity_alias'].apply(
    lambda x : leval(x) if x != 'no-alias' else x
)

In [80]:
wtr_trim_good['property_alias'] = wtr_trim_good['property_alias'].apply(
    lambda x : leval(x) if x != 'no-alias' else x
)

In [81]:
wtr_trim_good['object_alias'] = wtr_trim_good['object_alias'].apply(
    lambda x : leval(x) if x != 'no-alias' else x
)

In [82]:
wtr_trim_good.iloc[0]

reference_id                      390d6c6e68a32e11f8d7b0883cda0557db529fe6
reference_property_id                                                P4656
reference_datatype                                                     url
url                      https://en.wikipedia.org/w/index.php?title=G._...
netloc                                                    en.wikipedia.org
netloc_agg                                                en.wikipedia.org
final_url                https://en.wikipedia.org/w/index.php?title=G._...
html                         390d6c6e68a32e11f8d7b0883cda0557db529fe6.html
claim_id                     Q5512528$81E8AD02-28AF-4AE3-8ACD-047C30B40B01
rank                                                                normal
datatype                                                     wikibase-item
datavalue                {'value': {'entity-type': 'item', 'numeric-id'...
entity_id                                                         Q5512528
property_id              

# Getting Crowd Annotations

In [83]:
with open('../config/mongodb_credentials.json','r') as f:
    mongodb_credentials = json.load(f)

In [84]:
""" Connect to MTurk and to the Mongodb database. Set the boolean below to TRUE to use the marketplace and to FALSE to use the sandbox (testing the HITs)"""
create_hits_in_production = True
is_pilot = False

db_client = pymongo.MongoClient(mongodb_credentials["connection_string"])
db = db_client['textual_entailment']

collection_name = 'hit_results' + ('_sandbox' if not create_hits_in_production else '') + ('_pilots' if is_pilot else '')

hit_result_collection = db[collection_name]
hit_result_collection

Collection(Database(MongoClient(host=['cluster0-shard-00-00.hjstc.mongodb.net:27017', 'cluster0-shard-00-01.hjstc.mongodb.net:27017', 'cluster0-shard-00-02.hjstc.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='Cluster0-shard-0', ssl=True), 'textual_entailment'), 'hit_results')

In [85]:
from pprint import pprint
batch_pipeline = [
    {
        '$project': {
            'batch_id': 1, 
            'type': 1, 
            'scale': 1, 
            'hit': 1, 
            'timestamp': 1
        }
    }, {
        '$group': {
            '_id': '$batch_id', 
            'type': {
                '$max': '$type'
            }, 
            'scale': {
                '$max': '$scale'
            }, 
            'timestamp': {
                '$min': '$timestamp'
            }, 
            'ids': {
                '$addToSet': '$_id'
            }, 
            'reward': {
                '$avg': {
                    '$toDouble': '$hit.Reward'
                }
            }
        }
    }, {
        '$addFields': {
            'count': {
                '$size': '$ids'
            }
        }
    }
]
pprint(list(hit_result_collection.aggregate(batch_pipeline)))

[{'_id': 'd1d6e46a-d9e2-4b3e-93b9-c4b7d1ec414c',
  'count': 500,
  'ids': ['3P520RYKDKIKJDMC2BLU91K2QNL5UE',
          '3HEADTGN3S491F9A48828ORHWBIVR4',
          '3L2OEKSTXCMKN4PJAAPMO606O1X8YH',
          '373L46LKQAIENGWLXZLM4OU96WHJKE',
          '39XCQ6V3L1GZH2WB6TNRTV6MN1365D',
          '3Q2T3FD0PQKYSQUZFQ6F3U5OZ4QM3A',
          '3XU80RHWI2S36WOX90B2R5GURE244L',
          '37VUR2VJ7D1P914TFXYIC5LF7791CI',
          '3ULIZ0H1WDH4AGVE0NBAYZF5FQN15A',
          '33K3E8REXZ7VNZZ95E3K5STXMV38XB',
          '31GECDVAAMYV06WPBCXWL1KMJWE66T',
          '3G3AJKPCYO4J432HY7Z97UK1ME0Y4X',
          '3V8JSVE8Z121W5ZANDBMGGWSZ4VEYM',
          '3WGCNLZJLIKZETA7FTTTWB3XS7L1DI',
          '33KGGVH25XTDCZFO6TU5QI6P8PX1XK',
          '3Y40HMYLM4UTLW6M8H4P1KBMJLUXUD',
          '3ZURAPD29BZWCDO36ZOJKB6I4K11FT',
          '3YLPJ8OXYBPU4YC7TBD7OGDSK86X4Q',
          '3M7OI89LW10KGN5QL3AZ71CWFIPC6R',
          '3DW3BNF1HKU3JAUE5Z22H7WC6KV8V8',
          '3VAOOVPI424V2C23LK90Z3KGZLRLLL',
          '

In [86]:
sing = list(hit_result_collection.find(
    {'type':'single', 'batch_id' : {
        '$in' : ['d1d6e46a-d9e2-4b3e-93b9-c4b7d1ec414c']
    }}))
mult = list(hit_result_collection.find(
    {'type':'multiple', 'batch_id' : {
        '$in' : ['6a067c12-2f24-4024-af31-4d427b0a2c3b']
    }}))

TASK_HITS = {
    'single' : sing,
    'multiple' : mult
}
TASK_TYPES = list(TASK_HITS.keys())


len(sing), len(mult)

(500, 91)

In [87]:
from pprint import pprint

results_single = {
    'claim_id': [],
    'affirmation': [],
    'reference_id': [],
    'reference_url': [],
    'evidence_id': [],
    'sentence_id': [],
    'evidence': [],
    'score': [],
    'worker_id': [],
    'assignment_id': [],
    'relation': [],
    'reason_not_sure': [],
    'reason_not_sure_other': [],
    'times': []
}

for hit in TASK_HITS['single']:
    try:
        assert len(hit['answers']) == 5
        for answer in hit['answers']:
            for i, _ in enumerate(answer['values']['outputs']):
                subtask_info = hit['taskSet'][i]
                results_single['claim_id'].append(subtask_info['claim_id'])
                results_single['reference_id'].append(subtask_info['reference_id'])
                results_single['reference_url'].append(subtask_info['reference_url'])
                results_single['evidence_id'].append(subtask_info['evidence_id'])
                results_single['sentence_id'].append(subtask_info['sentence_id'])
                results_single['score'].append(subtask_info['score'])
                results_single['affirmation'].append(subtask_info['affirmation'])
                results_single['evidence'].append(subtask_info['evidence'])

                results_single['worker_id'].append(answer['worker_id'])
                results_single['assignment_id'].append(answer['assignment_id'])

                results_single['relation'].append(answer['values']['outputs'][i]['relation'])
                results_single['reason_not_sure'].append(answer['values']['outputs'][i]['reason_not_sure'])
                results_single['reason_not_sure_other'].append(answer['values']['outputs'][i]['reason_not_sure_other'])

                results_single['times'].append(answer['values']['times'][i])
    except Exception:
        print(hit['_id'])
        raise


results_single_df = pd.DataFrame(results_single)
assert results_single_df.shape[0] / 6 / 5 == len(TASK_HITS['single'])

In [88]:
from pprint import pprint

results_multiple = {
    'claim_id': [],
    'affirmation': [],
    'reference_id': [],
    'reference_url': [],
    'evidence_list': [],
    'worker_id': [],
    'assignment_id': [],
    'relation': [],
    'reason_not_sure': [],
    'reason_not_sure_other': [],
    'times': []
}

for hit in TASK_HITS['multiple']:
    try:
        #print(len(hit['answers']))
        assert len(hit['answers']) == 5
        for answer in hit['answers']:
            for i, _ in enumerate(answer['values']['outputs']):
                subtask_info = hit['taskSet'][i]
                results_multiple['claim_id'].append(subtask_info['claim_id'])
                results_multiple['reference_id'].append(subtask_info['reference_id'])
                results_multiple['reference_url'].append(subtask_info['reference_url'])
                results_multiple['affirmation'].append(subtask_info['affirmation'])
                results_multiple['evidence_list'].append(subtask_info['evidence_list'])

                results_multiple['worker_id'].append(answer['worker_id'])
                results_multiple['assignment_id'].append(answer['assignment_id'])

                results_multiple['relation'].append(answer['values']['outputs'][i]['relation'])
                results_multiple['reason_not_sure'].append(answer['values']['outputs'][i]['reason_not_sure'])
                results_multiple['reason_not_sure_other'].append(answer['values']['outputs'][i]['reason_not_sure_other'])

                results_multiple['times'].append(answer['values']['times'][i])
    except Exception:
        print(hit['_id'])
        raise


results_multiple_df = pd.DataFrame(results_multiple)
assert results_multiple_df.shape[0] / 6 / 5 == len(TASK_HITS['multiple'])

In [89]:
# Remember: 0=supports, 1=refutes, 2=neither, 3=notsure
def get_majority_relation(relations):
    relations = pd.Series(relations)
    relations_vc = relations.value_counts()
    
    majority_relation = -1
    tie = False
    
    if relations_vc.iloc[0] >= 3:
        # Cases 1-4
        majority_relation =  relations_vc.index[0]
        
    elif relations_vc.iloc[0] == 2:
        if relations_vc.iloc[1] == 1:
            # Case 5
            majority_relation =  relations_vc.index[0]
        else:
            # Case 6
            tie=True
            assert relations_vc.iloc[1] == 2
            # Implement tie table
            tie_1, tie_2 = relations_vc.index[0], relations_vc.index[1]
            if (tie_1, tie_2) in [(0,1), (1,0), (2,3), (3,2)]:
                majority_relation = 2
            elif (tie_1, tie_2) in [(0,2), (2,0), (0,3), (3,0)]:
                majority_relation = 0          
            elif (tie_1, tie_2) in [(1,2), (2,1), (1,3), (3,1)]:
                majority_relation = 1
            
    return majority_relation, tie

def get_majority_reason_not_sure(reasons):
    reasons = pd.Series(reasons)
    reasons_vc = reasons.value_counts()
    reasons_vc.drop(-1, inplace=True)
    if reasons_vc.size == 0:
        return -1
    return reasons_vc.index[0]

assert get_majority_relation([2,3,3,3,1]) == (3, False)
assert get_majority_relation([1, 1, 0, 0, 0]) == (0, False)
assert get_majority_relation([1, 1, 2, 2, 0]) == (1,True)


assert get_majority_reason_not_sure([-1, -1, -1, -1, -1]) == -1
assert get_majority_reason_not_sure([0, -1, -1, -1, -1]) == 0

In [176]:
results_single_df['reference_evidence_id'] = results_single_df.apply(
    lambda row : row['reference_id'] + '$' + str(row['evidence_id']), axis=1
)

results_single_df_agg = results_single_df.copy()
results_single_df_agg = results_single_df_agg\
    .groupby(['reference_evidence_id']).agg(list)

for c in results_single_df_agg.columns:
    if c in ['worker_id','assignment_id','relation','reason_not_sure','reason_not_sure_others','time']:
        continue
    else:
        if results_single_df_agg[c].map(lambda x : pd.Series(x).unique().size == 1).all():
            results_single_df_agg[c] = results_single_df_agg[c]\
                .map(lambda x : pd.Series(x).unique()[0])
            
            
relation_majs = results_single_df_agg['relation'].map(get_majority_relation)
results_single_df_agg['relation_maj'] = [x[0] for x in relation_majs]
results_single_df_agg['relation_maj_tie'] = [x[1] for x in relation_majs]
results_single_df_agg['reason_not_sure_maj'] = results_single_df_agg['reason_not_sure']\
    .map(get_majority_reason_not_sure)

results_single_df_agg = results_single_df_agg.reset_index(drop=True)
results_single_df_agg = results_single_df_agg.set_index(['reference_id','claim_id'])

results_single_df_agg = results_single_df_agg.drop(
    ['affirmation','reference_url','sentence_id','evidence_id','score'], axis=1
)

results_single_df_agg['evidence'] = results_single_df_agg['evidence'].apply(
    lambda x: x[0]
)

results_single_df_agg

evidence  \
reference_id                             claim_id                                                                                           
00d15820946d663523ad6ad9316fcc27ffe4a791 Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  Anaheim Ice, for­merly known as Disney Ice, is...   
                                         Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  In late 2019, the Ducks re­lo­cated their prac...   
                                         Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  It was the prac­tice and train­ing rink of the...   
                                         Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  It opened in 1995. When the Ana­heim Ducks (th...   
                                         Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  De­spite later re­sum­ing op­er­a­tions, Ana­h...   
...                                                                                                                                   ...   
fee28b055fe25d699a5092e83fdb146831cdefe1 Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  This episode marks Wesley Crusher's final regu...   
                                         Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  He later appeared in the fifth season Next Gen...   
                                         Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  Brent Spiner as Lt. Commander Data. Wil Wheato...   
                                         Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  After Picard leaves a directional marker on th...   
                                         Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  The USS Enterprise-D rendezvouses with Captain...   

                                                                                                                                worker_id  \
reference_id                             claim_id                                                                                           
00d15820946d663523ad6ad9316fcc27ffe4a791 Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  [A1KL6MRIHQFJND, A3BHKF9NJKCXPB, A3O0NNIW4KA2P...   
                                         Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  [A1SSQSHIYUNDZ0, A9LXSGXZM4IJE, A12BUI6X0OB6SY...   
                                         Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  [A2R05XFXQLZEC0, A3O0NNIW4KA2PV, A2RKUDGK5PQ44...   
                                         Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  [A25SH3L1UR6Y1Z, A2OI65EUOJ0CBY, A2HUR35MRR4DY...   
                                         Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  [A7EP668Z0WF6G, A3LCGX74R6XR5I, A1PO4Z1B8661LE...   
...                                                                                                                                   ...   
fee28b055fe25d699a5092e83fdb146831cdefe1 Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  [A2RKUDGK5PQ44X, AHYTT3T8W9Y9Y, A1EQ8UXKOUDYT4...   
                                         Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  [A7EP668Z0WF6G, A3EZIJXR1OISKT, A3UX5CA3HXK7FP...   
                                         Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  [A48WNR6C4CI3J, AT2TNRST7C15B, A2RKUDGK5PQ44X,...   
                                         Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  [AT2TNRST7C15B, A3O0NNIW4KA2PV, A1DZMZTXWOM9MR...   
                                         Q5449415$E949A455-C67B-4F85-AB57-B99A3F4C4FBD  [A2I6ZALE49CVSC, A1PO4Z1B8661LE, AB7ZGKRL87OZ1...   

                                                                                                                            assignment_id  \
reference_id                             claim_id                                                                                           
00d15820946d663523ad6ad9316fcc27ffe4a791 Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7  [3KGTPGBS61XV701JAUOY0EDJ0STU2J, 386CSBG1O3YRX...   
                                      

In [177]:
wtr_trim_good['crowd_annotations_T1'] = wtr_trim_good.apply(
    lambda x : results_single_df_agg.loc[
        x['reference_id'],x['claim_id']
    ].to_dict(orient='records'), axis=1
)

In [181]:
results_multiple_df_agg = results_multiple_df.copy()
results_multiple_df_agg = results_multiple_df_agg\
    .groupby(['reference_id']).agg(list)
results_multiple_df_agg.drop(
    ['evidence_list'], axis=1, inplace=True
)

for c in results_multiple_df_agg.columns:
    if c in ['worker_id','assignment_id','relation','reason_not_sure','reason_not_sure_others','time']:
        continue
    else:
        if results_multiple_df_agg[c].map(lambda x : pd.Series(x).unique().size == 1).all():
            results_multiple_df_agg[c] = results_multiple_df_agg[c]\
                .map(lambda x : pd.Series(x).unique()[0])
            
relation_majs = results_multiple_df_agg['relation'].map(get_majority_relation)
results_multiple_df_agg['relation_maj'] = [x[0] for x in relation_majs]
results_multiple_df_agg['relation_maj_tie'] = [x[1] for x in relation_majs]
results_multiple_df_agg['reason_not_sure_maj'] = results_multiple_df_agg['reason_not_sure']\
    .map(get_majority_reason_not_sure)

In [182]:
results_multiple_df_agg = results_multiple_df_agg.reset_index()


results_multiple_df_agg = results_multiple_df_agg.drop(
    ['affirmation','reference_url'], axis=1
)

results_multiple_df_agg['evidence'] = results_multiple_df_agg.apply(
    lambda x : list(results_single_df_agg.loc[
        x['reference_id'], x['claim_id']
    ]['evidence']), axis=1
)

results_multiple_df_agg = results_multiple_df_agg.set_index(['reference_id','claim_id'])


results_multiple_df_agg.head()

,,worker_id,assignment_id,relation,reason_not_sure,reason_not_sure_other,times,relation_maj,relation_maj_tie,reason_not_sure_maj,evidence
reference_id,claim_id,,,,,,,,,,
00d15820946d663523ad6ad9316fcc27ffe4a791,Q4750883$C5B95856-A961-4EA4-95F5-995B94509CC7,"[A1SAAR4KRNSLKQ, A2D74QPANDB2I7, A1EQ8UXKOUDYT...","[3JV9LGBJWXQQ9PPTDNXE3CFVSQCGOF, 3AZHRG4CU8W4C...","[1, 0, 0, 2, 0]","[-1, -1, -1, -1, -1]","[, , , , ]","[6920, 43217, 23941, 10066, 70548]",0,False,-1,"[Anaheim Ice, for­merly known as Disney Ice, i..."
00d4fa84c656b876dddecc868489d261554c4ca1,Q17849283$4EDA0CCF-7158-485C-9D9C-28326CAEC967,"[A1B1DPJRVIAV3L, A39MKVROUZ1UWR, AZ5ZYUCAQ0XDL...","[3E337GFOLDK8R4PHC8X1ONFQ2G7GN0, 3SLE99ER0RP6H...","[2, 2, 2, 2, 0]","[-1, -1, -1, -1, -1]","[, , , , ]","[27871, 38728, 20531, 24575, 6573]",2,False,-1,[Description. Two bascule bridges of timber co...
00d94a4a5c4ea458a8abe9a15682a4499114b634,Q17777953$c15f71a3-4c08-3798-78dd-a4ed158a5286,"[AZ5ZYUCAQ0XDL, A1RGKVJZAFBN4Q, A1EQ8UXKOUDYT4...","[3QY5DC2MXVWF8UDVYP35KX789X0UF7, 3X73LLYYQ5QMR...","[2, 0, 2, 2, 1]","[-1, -1, -1, -1, -1]","[, , , , ]","[58603, 56068, 35421, 4366, 58321]",2,False,-1,[Historic Environment Scotland is responsible ...
0280723077b22b8f9cf2c636fa889db510e0272d,Q56818480$2D60B386-F18B-43B3-9B76-C5B4BE960D2A,"[A219SZBLCCCXN9, A3774HPOUKYTX7, AB7ZGKRL87OZ1...","[3483FV8BEIVA5PBIECQVK8Z3X7E26U, 337RC3OW09E1L...","[0, 0, 0, 2, 2]","[-1, -1, -1, -1, -1]","[, , , , ]","[38893, 12117, 74042, 110405, 78648]",0,False,-1,"[Appropriation Act (No. 1) 1990-91., 750 000 4..."
0427581cda67794db9cab34a1f91379a8004acbd,Q76051026$0ad99c6c-43fb-846f-5698-dcc435710ae1,"[ATBBPV37O4K6N, A1MXDBILWV8RXN, ACOGH7IWJZR6A,...","[3M1CVSFP64HSKAP660X8WVZS9WIQAM, 3HSYG7LRBNACL...","[1, 2, 1, 1, 0]","[-1, -1, -1, -1, -1]","[, , , , ]","[3256, 86417, 61207, 58797, 61562]",1,False,-1,"[Sir Noel Short, who has died aged 85, was Sec..."


In [184]:
wtr_trim_good['crowd_annotations_T2'] = wtr_trim_good.apply(
    lambda x : results_multiple_df_agg.loc[
        x['reference_id'],x['claim_id']
    ].to_dict(), axis=1
)

# Author Annotations

In [190]:
wtr_trim_good.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409 entries, 0 to 408
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   reference_id           409 non-null    object
 1   reference_property_id  409 non-null    object
 2   reference_datatype     409 non-null    object
 3   url                    409 non-null    object
 4   netloc                 409 non-null    object
 5   netloc_agg             409 non-null    object
 6   final_url              409 non-null    object
 7   html                   409 non-null    object
 8   claim_id               409 non-null    object
 9   rank                   409 non-null    object
 10  datatype               409 non-null    object
 11  datavalue              409 non-null    object
 12  entity_id              409 non-null    object
 13  property_id            409 non-null    object
 14  entity_label           409 non-null    object
 15  property_label         

In [195]:
author_ann = pd.read_csv('../data/unique_references_for_manual_annotation.csv')
author_ann = author_ann.drop('index', axis=1)
author_ann = author_ann.set_index(['reference_id','claim_id'])
author_ann

,,affirmation,entity_label,property_label,object_label,final_url,manual_label
reference_id,claim_id,,,,,,
5b14a71742a09446d03a95c9426a659cda3bad6f,Q62637091$C1F95944-8A83-4901-80FF-931DB0F448EC,Priscila Farias was born in 1964.,Priscila Farias,date of birth,1964,https://www.lambiek.net/artists/f/farias_prisc...,1.A.
15bea5900b8dda1bcf6f622348d80ef5a5b7b71e,Q5798066$60E7A7BB-9549-4D96-8395-A4875F72E5C5,Daniel Diaz was born on 14/09/1966.,Daniel Diaz,date of birth,14/09/1966,https://www.lambiek.net/artists/d/dani-the-o.htm,1.B.
74188de4f031ca41e7927959f818816dbdfc29af,Q99607134$2818F414-8F79-4677-8D9F-AD9B7E977A4A,The Value Added Tax (Section 55A) (Specified S...,The Value Added Tax (Section 55A) (Specified S...,publication date,2016,https://www.bailii.org/uk/legis/num_reg/2016/u...,1.B.
11513fdd381730dfa38bf9fb5a724a129dc8480c,Q99686151$4E1BDE62-B95E-4C12-92F2-192FB49E6449,The Proscribed Organisations (Name Changes) Or...,The Proscribed Organisations (Name Changes) Or...,publication date,2011,https://www.bailii.org/uk/legis/num_reg/2011/u...,1.B.
99415d84e2b5632a86eaa925c4f7ae791816621e,Q4815670$2BFBD30C-D13D-4A04-A76C-F4C60574852B,Athos Bulcão died on 31/07/2008.,Athos Bulcão,date of death,31/07/2008,http://archive.ph/m6yCn,2.B.
...,...,...,...,...,...,...,...
c30a95bb7d44805e1d654b05b5f0bc31a49d8123,Q14650496$409733e3-46f4-7c38-ceaf-cb5e4f40c573,Furious 7 is an action film.,Furious 7,genre,action film,http://archive.ph/Pf2Dq,1.B.
d2adcd8a228546ea69fec3002a066217cce0e97b,Q64460267$F6CEBC9F-F532-403F-83E9-CB2CD486C1D8,Naravoor Lps is located in Kannur district.,Naravoor Lps,is located in,Kannur district,https://schools.org.in/Kannur/32020700608/,1.A.
080da364904fb2c94058b444213026d02570411d,Q63156797$F98C48C4-F906-4958-82A1-6E0EA56EDE5E,William Welham Clarke died in 1877.,William Welham Clarke,date of death,1877,https://www.gracesguide.co.uk/William_Welham_C...,1.A.


In [197]:
wtr_trim_good['author_annotations'] = wtr_trim_good.apply(
    lambda x : author_ann.loc[
        x['reference_id'],x['claim_id']
    ]['manual_label'], axis=1
)

In [201]:
wtr_trim_good.to_json('./WTR/WTR.json', orient='records')